In [4]:
#!pip install langchain langchainhub langchain_openai openai

from langchain.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain import hub
#from google.colab import userdata
from dotenv import load_dotenv
import os
load_dotenv()
secret_string = os.getenv("OPENAI_API_KEY")

# Inicializamos db
db = SQLDatabase.from_uri("sqlite:///chinook.db")
# Pull down prompt
prompt = hub.pull("rlm/text-to-sql")
# Initialize model
model = ChatOpenAI(openai_api_key=secret_string)

# Crear chain con LangChain Expression Language
inputs = {
    "table_info": lambda x: db.get_table_info(),
    "input": lambda x: x["question"],
    "few_shot_examples": lambda x: "",
    "dialect": lambda x: db.dialect,
}
sql_response = (
    inputs
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

# Call with a given question
sql_response.invoke({"question": "Enumera las ventas totales por país. ¿Los clientes de qué país gastaron más?"})

'SQLQuery: \nSELECT invoices.BillingCountry, SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS TotalSales\nFROM invoices\nJOIN invoice_items ON invoices.InvoiceId = invoice_items.InvoiceId\nGROUP BY invoices.BillingCountry\nORDER BY TotalSales DESC\nLIMIT 1\n'

In [2]:
!pip install python-dotenv